In [1]:
from scr.utils import read_CLINC150_file, get_label_set, turn_single_label_to_multilabels

dataDict = read_CLINC150_file('data/CLINC150/data_full.json')
trainList = dataDict['train']
valList = dataDict['val']
testList = dataDict['test']
labelSet = get_label_set(trainList, valList, testList)

turn_single_label_to_multilabels(trainList, testList, valList)

d:\Projects\BertClassifier\venv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


The keys found in this json are:  dict_keys(['oos_val', 'val', 'train', 'oos_test', 'test', 'oos_train'])


In [2]:
from scr.dataset import SentenceLabelDataset

trainSet = SentenceLabelDataset(trainList, labelSet)

In [3]:
from torch.utils.data import DataLoader
from scr.dataset import collate_dynamic_padding

dataloader = DataLoader(trainSet, 
                        batch_size = 8, 
                        num_workers = 2, 
                        shuffle = True,
                        collate_fn = collate_dynamic_padding,
                        pin_memory = True)

In [7]:
from scr.evaluate import MultiLabelEvaluator
import torch


FUNCTION_MAP = {'accuracy' : MultiLabelEvaluator.get_accuracy,
                'precision' : MultiLabelEvaluator.get_precision,
                'recall' : MultiLabelEvaluator.get_recall,
                'f1' : MultiLabelEvaluator.get_F1}

filter = ['Accuracy']

targets = [[1,0,1], [1,0,0], [1,1,0],[1,1,1]]
preds = [[1,0,0], [0,1,0], [1,1,0], [1,0,0]]
targets = torch.as_tensor(targets)
preds = torch.as_tensor(preds)


funs = [FUNCTION_MAP[fil.lower()] for fil in filter]
evaluator = MultiLabelEvaluator()
evaluator.add_batch(probs = preds, targets=targets)
for fun in funs:
    print(fun(evaluator))


(tensor([0.7500, 0.3333, 0.0000]), 0.4444444477558136)


In [4]:
from torch.nn import Sigmoid
import torch

temp = [[15,2,-1],[1,4,6]]
temp = torch.as_tensor(temp)

sigmoid = Sigmoid()
print(sigmoid(temp))

tensor([[1.0000, 0.8808, 0.2689],
        [0.7311, 0.9820, 0.9975]])


In [3]:
evaluator.truePositive

tensor([[1, 0, 0],
        [0, 0, 0],
        [1, 1, 0],
        [1, 1, 0]], dtype=torch.int32)

In [5]:
'Accuracy'.lower()

'accuracy'

In [11]:
torch.where(evaluator.truePositive==0, preds, 0)

tensor([[0, 0, 0],
        [0, 1, 0],
        [0, 0, 0],
        [0, 0, 0]])

In [4]:
evaluator.positive

tensor([[1, 0, 1],
        [1, 1, 0],
        [1, 1, 0],
        [1, 1, 1]], dtype=torch.int32)

In [46]:
print(preds)
print(preds[:,2])


tensor([[1, 1, 0],
        [0, 0, 1],
        [0, 0, 1],
        [0, 0, 1]], dtype=torch.int32)
tensor([0, 1, 1, 1], dtype=torch.int32)


In [1]:
from scr.dataset import  SentenceLabelDataset
from transformers import BertTokenizer
from scr.models import BertMLPClassifier
from scr.utils import load_checkpoint, read_CLINC150_file, get_label_set, turn_single_label_to_multilabels
import torch

c:\Users\18520\anaconda3\envs\rasa1\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataPath = 'data/CLINC150/data_oos_plus.json'
dataDict = read_CLINC150_file(dataPath)

device = torch.device('cpu')
trainList = dataDict['train'] + dataDict['oos_train']
valList = dataDict['val'] + dataDict['oos_val']
testList = dataDict['test'] + dataDict['oos_test']

turn_single_label_to_multilabels(trainList, valList, testList)
labelSet = get_label_set(trainList, valList, testList)

trainSet = SentenceLabelDataset(trainList, labelSet, tokenizer = BertTokenizer.from_pretrained('bert-base-uncased'))
tokenizer = trainSet.tokenizer

modelPath = 'model/best.pth.tar'
model = BertMLPClassifier()
# model.to(device)
load_checkpoint(modelPath, model,device=device)

The keys found in this json are:  dict_keys(['oos_val', 'val', 'train', 'oos_test', 'test', 'oos_train'])


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


=> loaded checkpoint 'True' (epoch 219)
Checkpoint's train loss is: 0.0071
Checkpoint's validation loss is: 0.0101


(219, 0.007136629820646097, 0.010111123584210872)

In [9]:
texts = ['tell me my bank balance',
         'tranfer the money into my deposit account',
         'how to count to 10 in german',
         'how to say goodbye in spanish', 
         'see you later',
         'tell me a joke',
         'where the nearest convinience store ?', 
         'how to make egg fried rice',
         'who was geogre washington',
         'withdraw $500 from my saving account',]

In [10]:
from scr.utils import predict_class
predict_class(texts, model, tokenizer, labelSet)

['balance',
 'order_checks',
 'translate',
 'translate',
 'goodbye',
 'tell_joke',
 'directions',
 'recipe',
 'oos',
 'transfer']

In [6]:
len(labelSet)

151